In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import lightgbm as lgb
# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestRegressor,  
                               ExtraTreesRegressor)
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [2]:
%%time
df_train = pd.read_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/ultimate2/train_test/train_features.csv")
df_test = pd.read_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/ultimate2/train_test/test_features.csv")

Wall time: 9.05 s


In [3]:
print(df_train.shape)
df_test.shape

(201917, 86)


(123623, 84)

In [4]:
def feature_concat(train_data1,filepath):
    for file in filepath:
        data_features = pd.read_csv(file)
        data_features = data_features.drop_duplicates()
        train_data1=pd.merge(train_data1,data_features,how='left',on='card_id')
#         print(file)
#         print(train_data1.shape)
    return(train_data1)

In [5]:
%%time
import glob
file_list = glob.glob("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/features_90/features_80/*.csv")
train_features=feature_concat(df_train,file_list)
test_features=feature_concat(df_test,file_list)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']



C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (1,9,17,28,37,44,55,64,70,78,87,94,104,113,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Wall time: 3min 24s


In [6]:
print(train_features.shape)
test_features.shape

(201917, 534)


(123623, 532)

In [7]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
target = df_train['target']
train_features=train_features.fillna(0)
test_features=test_features.fillna(0)

In [8]:
#Some useful parameters which will come in handy later on
ntrain = train_features.shape[0]
ntest = test_features.shape[0]
SEED = 4590 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)


# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [9]:
def get_oof(clf, df_train, y_train, x_test,df_train_columns):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    val_metrics=[]
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
        print("fold {}".format(fold_))
        x_tr = df_train.iloc[trn_idx][df_train_columns]
        y_tr = target.iloc[trn_idx]
        x_va = df_train.iloc[val_idx][df_train_columns]
        y_va = target.iloc[val_idx]
        clf.train(x_tr, y_tr)
        oof_train[val_idx] = clf.predict(x_va)
        oof_test_skf[fold_, :] = clf.predict(x_test[df_train_columns])
        val_metrics.append(np.sqrt(mean_squared_error(clf.predict(x_va), y_va)))
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1),val_metrics
        


In [10]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'max_depth': 60,
    'min_samples_leaf': 177,
    'verbose': 1
}

lgb_param1 = {
    'max_leaves':100,
             'min_data_in_leaf': 177, 
             'max_depth': 60,
            'feature_fraction':0.8,
             'objective':'regression',
             'learning_rate': 0.01,
             "boosting": "gbdt",
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": 4,
             "random_state": 4590}

lgb_param2=lgb_param1.copy()
lgb_param2['type']='gamma'

lgb_param3=lgb_param1.copy()
lgb_param3['type']='tweedie'



# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'max_depth': 60,
    'min_samples_leaf' : 177,
    'verbose': 0
}




In [11]:
# Create object
rf = SklearnHelper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesRegressor, seed=SEED, params=et_params)
# ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
# gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
# svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)
# xgbo = SklearnHelper(clf=xgb1, seed=SEED, params=svc_params)
# lgbo = SklearnHelper(clf=lgb, seed=SEED, params=svc_params)

In [12]:
# # important Features
#step1:
train_features = train_features.select_dtypes(include=numerics)
train_features.columns = [x.replace("<lambda>","range") for x in train_features.columns]
test_features.columns = [x.replace("<lambda>","range") for x in test_features.columns]
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers','class_outcome']]

# step2:
# df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]


# step3:
#df_train_columns = imp_features1

# step4:
# df_train_columns=df_train_columns.extend('class_outcome')


In [13]:
et_oof_train, et_oof_test,val_metrics_et = get_oof(et, train_features, target, test_features,df_train_columns) # Extra Trees
print(val_metrics_et)
rf_oof_train, rf_oof_test,val_metrics_rf = get_oof(rf, train_features, target, test_features,df_train_columns)# Random Forest
print(val_metrics_rf)
# ada_oof_train, ada_oof_test,val_metrics_ada = get_oof(ada, train_features, target, test_features,train_features_columns) # AdaBoost 
# gb_oof_train, gb_oof_test,val_metrics_gb = get_oof(gb, train_features, target, test_features,train_features_columns) # Gradient Boost
# svc_oof_train, svc_oof_test,val_metrics_svc = get_oof(svc, train_features, target, test_features,df_train_columns) # Support Vector Classifier

fold 0
fold 1
fold 2
fold 3
fold 4


KeyboardInterrupt: 

In [44]:
np.mean([0.09898099076058094, 0.0994351621675069, 0.09939453033492776, 0.09977368273143244, 0.10005657266392057])

0.09952818773167374

### Light GBM

In [42]:
### Lightgbm
def run_lgb1(df_train,df_test,df_train_columns,lgb_params):
    param =  lgb_params
    val_metrics=[]
    oof_test_skf = np.empty((5, len(df_test)))
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
    oof_train = np.zeros(len(df_train))
    oof_test = np.zeros(len(df_test))
    feature_importance_df = pd.DataFrame()
    print(df_train.shape)
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)
#         print(trn_data.shape,val_data.shape)
        num_round = 3000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
        oof_train[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
#
        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = df_train_columns
        fold_importance_df["importance"] = clf.feature_importance()
        
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        test_predictions = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)
        oof_test_skf[fold_, :] = test_predictions
        
        predictions= clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
        val_metrics.append(np.sqrt(mean_squared_error(predictions, target.iloc[val_idx])))
    oof_test[:] = oof_test_skf.mean(axis=0)
    return feature_importance_df,oof_train.reshape(-1,1),oof_test.reshape(-1,1),val_metrics

In [43]:
imp_features,lgb_oof_train, lgb_oof_test,val_metrics=run_lgb1(train_features,test_features,df_train_columns, lgb_param1)
print(val_metrics)

(201917, 377)
fold 0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0986163	valid_1's rmse: 0.100066
[200]	training's rmse: 0.096277	valid_1's rmse: 0.0992515
[300]	training's rmse: 0.0945439	valid_1's rmse: 0.0990469
[400]	training's rmse: 0.0931198	valid_1's rmse: 0.0989835
[500]	training's rmse: 0.0919433	valid_1's rmse: 0.0990156
Early stopping, best iteration is:
[431]	training's rmse: 0.0927349	valid_1's rmse: 0.098981
fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0985026	valid_1's rmse: 0.100304
[200]	training's rmse: 0.0961795	valid_1's rmse: 0.0995641
[300]	training's rmse: 0.0944359	valid_1's rmse: 0.0994464
[400]	training's rmse: 0.0930167	valid_1's rmse: 0.0994472
Early stopping, best iteration is:
[355]	training's rmse: 0.0936215	valid_1's rmse: 0.0994352
fold 2
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0985629	valid_1's rmse: 0.100337
[20

In [35]:
# imp_features1 = imp_features.groupby('Feature').agg({'importance':sum}).reset_index().sort_values(by = 'importance', ascending = False).head(250)['Feature'].tolist()

In [48]:
x_train = np.concatenate((et_oof_train, rf_oof_train,lgb_oof_train), axis=1)
y_train = target
x_test = np.concatenate((et_oof_test, rf_oof_test,lgb_oof_test), axis=1)

### Stacking 

In [49]:
### Lightgbm
def run_lgb2(df_train, df_test, lgb_params):
   param =  lgb_params
   val_metrics=[]
   oof_test_skf = np.empty((5, len(df_test)))
   folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
   oof_train = np.zeros(len(df_train))
   oof_test = np.zeros(len(df_test))
   #feature_importance_df = pd.DataFrame()
   print(df_train.shape)
   for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,target)):
       print("fold {}".format(fold_))
       trn_data = lgb.Dataset(df_train.iloc[trn_idx], label=y_train.iloc[trn_idx])#, categorical_feature=categorical_feats)
       val_data = lgb.Dataset(df_train.iloc[val_idx], label=y_train.iloc[val_idx])#, categorical_feature=categorical_feats)
#         print(trn_data.shape,val_data.shape)
       num_round = 3000
       clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
       oof_train[val_idx] = clf.predict(df_train.iloc[val_idx], num_iteration=clf.best_iteration)
#
#        fold_importance_df = pd.DataFrame()
#        fold_importance_df["Feature"] = df_train_columns
       #fold_importance_df["importance"] = clf.feature_importance()
       
       #fold_importance_df["fold"] = fold_ + 1
       #feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

       test_predictions = clf.predict(df_test, num_iteration=clf.best_iteration)
       oof_test_skf[fold_, :] = test_predictions
       
       predictions= clf.predict(df_train.iloc[val_idx], num_iteration=clf.best_iteration)
       val_metrics.append(np.sqrt(mean_squared_error(predictions, y_train.iloc[val_idx])))
   oof_test[:] = oof_test_skf.mean(axis=0)
   return oof_train.reshape(-1,1),oof_test.reshape(-1,1),val_metrics
# stack_train=pd.DataFrame(x_train)
# target=train_features['outliers']

In [50]:
stack_train=pd.DataFrame(x_train)
target=train_features['outliers']
lgb_oof_train, lgb_oof_test,val_metrics=run_lgb2(stack_train, pd.DataFrame(x_test), lgb_param1)

(201917, 3)
fold 0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0999334	valid_1's rmse: 0.100048
[200]	training's rmse: 0.0991579	valid_1's rmse: 0.0994245
[300]	training's rmse: 0.0988866	valid_1's rmse: 0.0993505
[400]	training's rmse: 0.0987026	valid_1's rmse: 0.0994011
Early stopping, best iteration is:
[300]	training's rmse: 0.0988866	valid_1's rmse: 0.0993505
fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0998354	valid_1's rmse: 0.10026
[200]	training's rmse: 0.0990506	valid_1's rmse: 0.0997645
[300]	training's rmse: 0.0987885	valid_1's rmse: 0.0997438
Early stopping, best iteration is:
[254]	training's rmse: 0.0988846	valid_1's rmse: 0.0997369
fold 2
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0998456	valid_1's rmse: 0.100169
[200]	training's rmse: 0.099046	valid_1's rmse: 0.0996682
[300]	training's rmse: 0.0987672	valid_1's rmse: 0.0996521
Early

In [51]:
np.mean(val_metrics)

0.09978908168569459

In [52]:
submission=pd.DataFrame(lgb_oof_test)
submission['card_id']=test_features['card_id']
submission.columns=['target','card_id']
submission=submission[['card_id','target']]
submission.to_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/tf_combo2.csv",index=False)
# test_class.to_csv("../fresh_code/ultimate2/features/class/test_class.csv",index=False)

In [53]:
submission.shape

(123623, 2)

In [54]:
stack_test = pd.DataFrame(x_test)
stack_test['card_id'] = test_features['card_id']
stack_test.to_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/tfs_combo2.csv",index=False)
